In [1]:
using DataFrames
using Distributions
using Plots
pyplot()

Plots.PyPlotBackend()

In [2]:
function generate_data(n1,n2, d=2)
    # Returns 2 clouds of labelled Gaussian data of dim 2 with different moments
    model1 = MvNormal(-ones(d), .5*eye(d))
    model2 = MvNormal(ones(d), .5*eye(d))
    samples1 = rand(model1, n1)
    samples2 = rand(model2, n2)
    X = DataFrame([transpose(samples1); transpose(samples2)])
    X = hcat(X, zeros(n1+n2))
    y = DataFrame(Label=ones(size(X,1)))
    y[n1+1:n1+n2,:Label] = -1*ones(n2)
    return X, y
end

generate_data (generic function with 2 methods)

In [3]:
function transform_svm_primal(X, y, C = .5)
    d = size(X,2)
    n = size(X,1)
    Q = zeros(d+n, d+n)
    Q[1:d, 1:d] = eye(d)
    p = zeros(d+n)
    p[d+1:d+n] = [C for _ in 1:n]
    Y = diagm(y[:Label])
    A = zeros(n, d+n)
    A[1:n, d+1:d+n] = -eye(n)
    A[1:n,1:d] = - Y*Array(X)
    b = -ones(n)
    return Q, p, A, b
end

transform_svm_primal (generic function with 2 methods)

In [4]:
function transform_svm_dual( X, y,C = .5)
    d = size(X,2)
    n = size(X,1)
    Q = diagm(y[:Label]) * Array(X) * transpose(Array(X)) * diagm(y[:Label])
    p = ones(n)
    A = zeros(2*n, n)
    b = zeros(2*n)
    A[1:n, 1:n] = eye(n)
    A[n+1:2n, 1:n] = -eye(n)
    b[1:n] = C*ones(n)
    return Q, p, A, b
end

transform_svm_dual (generic function with 2 methods)

In [39]:
function Newton(t, x, f, grad, hess)
    invHess = pinv(hess(x,t)) 
    nabla = grad(x,t)
    dx = - invHess * nabla
    lam2 = - transpose(dx) * nabla
    backtrack_t = backtracking(x, t, dx, f, nabla)
    x_new = x + backtrack_t*dx
    gap = lam2/2
    return x_new, gap[1]
end

function backtracking(x, t, dx, f, nabla)
    alpha = .25 
    beta = .5
    res = 1
    while minimum(x + res*dx) < 1
        res *= beta
    end
    while  f(x+res*dx, t) - f(x,t) + (alpha * res*transpose(nabla)*dx)[1] > 0.
        res *= beta
    end
    return res
end

backtracking (generic function with 1 method)

In [40]:
function centering_step(Q, p, A, b, x, t=1, tol=10^(-3.))
    function f(u,v)
        return (v * (transpose(u) * Q * u + transpose(p)*u) - sum(log(b - A * u)))[1]
    end
    function grad(u,v)
        return v * (Q * u + p) + sum([transpose(A[i,:])/(b[i] - A[i,:] * u)[1] for i in 1:size(A,1)])
    end
    function hess(u,v)
        return v * Q - sum([transpose(A[i,:])*A[i,:]/(b[i] - A[i,:] * u)[1]^2. for i in size(A,1)])
    end
    x_seq = []
    gaps = []
    gap = Inf
    x_new = x
    while (gap > tol) 
        x_new, gap = Newton(t, x_new, f, grad, hess)
        push!(x_seq, x_new)
        push!(gaps, gap)
    end
    return x_seq
end

centering_step (generic function with 3 methods)

In [41]:
function barr_method(Q, p, A, b, x_0, mu, tol=10^(-3.))
    x = x_0
    m = size(x_0,1)
    t = 1
    gaps = []
    while true        
        push!(gaps, m/t)
        x = centering_step(Q, p, A, b, x, t, tol)
        t = mu * t
        if m/t < tol
            x_sol = x
            break
        end
    end
    plot(gaps)
    return x_sol, x_seq
end

barr_method (generic function with 2 methods)

In [42]:
function SVM_primal(X, y, C = .5)
    n, d = size(X)
    Q, p, A, b = transform_svm_primal(X,y,C)
    mu = 2
    x_0 = C/2 * ones(size(A,2))
    barr_method(Q,p,A,b, x_0, mu)
end

SVM_primal (generic function with 2 methods)

In [43]:
function SVM_dual(X, y, C=.5)
    Q, p, A, b = transform_svm_dual(X, y,C)
    mu = 2
    x_0 = C/2 * ones(size(A,2))
    barr_method(Q,p,A,b, x_0, mu)
end

SVM_dual (generic function with 2 methods)

In [ ]:
X,y = generate_data(20,20)
C = .5
SVM_primal(X,y,C)

In [24]:
X,y = generate_data(20,20)
C = .5
Q, p, A, b = transform_svm_dual(X,y,C)
x= C/2 * ones(size(A,2))
function f(u,v)
    return v * (transpose(u) * Q * u + transpose(p)*u)[1]
end
f(x,1)

197.40575106517804